In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In this notebook I am looking at one flying ball. I selected a play that is a deep kickback and I look at the speed and acceleration of the flying ball. 
Normally, after the kick, the acceleration of the ball should be constant and equal to the gravitational acceleration, if the acceleration from the tracking data is given by an accelerometer attached to the ball. The speed in the plane of the field should be constant. Weirdly, this is not what we see in the data. 

In [ ]:
# Read in data files
BASE_DIR = '../input/nfl-big-data-bowl-2022'
players = pd.read_csv(f'{BASE_DIR}/players.csv')
PFFScouting = pd.read_csv(f'{BASE_DIR}/PFFScoutingData.csv')
tracking2019 = pd.read_csv(f'{BASE_DIR}/tracking2019.csv')
tracking2020 = pd.read_csv(f'{BASE_DIR}/tracking2020.csv')
tracking2018 = pd.read_csv(f'{BASE_DIR}/tracking2018.csv')
games = pd.read_csv(f'{BASE_DIR}/games.csv')
plays = pd.read_csv(f'{BASE_DIR}/plays.csv')

In [ ]:
import matplotlib.pylab as plt

Here we already find a play with a deep kick from J. Elliott. We are just going to look at this play in the tracking data. 

In [ ]:
plays.head(3)

The hang time is 3.85s

In [ ]:
PFFScouting.query('gameId == 2018090600 & playId == 37' )['hangTime']

In [ ]:
tr_ball = tracking2018.query('gameId == 2018090600 & playId == 37 & team == "football"').reset_index(drop = True)

In [ ]:
set(tr_ball['event'].values)

In [ ]:
tr_ball.query('event == "kickoff" or event == "touchback"')

Here I make a column called gtime (good time) where the time is expressed in seconds from teh kickoff

In [ ]:

tr_ball['gtime'] = tr_ball['time'].str.split(":").str[-1].astype('float32')
T0 = tr_ball.query('event == "kickoff"')['gtime'].reset_index(drop = True)[0] 
tr_ball['gtime'] = tr_ball['gtime'] - T0

In [ ]:
# here I add a column with the calculated speed in the x direction
tr_ball['sx'] = tr_ball['s'] 
for i in range(1,len(tr_ball)):
    tr_ball.at[i,'sx'] = (tr_ball.at[i,'x'] - tr_ball.at[i-1,'x'])/(tr_ball.at[i,'gtime'] - tr_ball.at[i-1,'gtime'])

In [ ]:
# here I add a column with the calculated speed in the y direction
tr_ball['sy'] = tr_ball['s'] 
for i in range(1,len(tr_ball)):
    tr_ball.at[i,'sy'] = (tr_ball.at[i,'y'] - tr_ball.at[i-1,'y'])/(tr_ball.at[i,'gtime'] - tr_ball.at[i-1,'gtime'])

In [ ]:
# ready for plotting, let's just turn all these into numpy arrays
#acceleration
p_a = tr_ball[['gtime', 'a']].to_numpy()
#speed
p_s = tr_ball[['gtime', 's']].to_numpy()
#calculated speed along x
p_sx = tr_ball[['gtime', 'sx']].to_numpy()
#calculated speed along y
p_sy = tr_ball[['gtime', 'sy']].to_numpy()

Now we plot s and s_x. s is the speed as read from the tracking data, s_x is the speed along x as calculated from the tracking data. 
s_x should really be a constant until the moment the ball touches the ground or another player. It is not. Why is s_x changing while the ball flies? According to the laws of kinematis as long as the ball is in the air, the acceleration in the x-y plane should be zero, therefore the speeds along x and y should be constant.
The speed s from the trackind data seems to follow quite closely the speed calculated from the x displacement vs time. This indicates that the speed is not some three dimensional speed, but a speed in the x-y plane, as the one for the players.


In [ ]:
# let's plot s and s_x

plt.plot(p_s[:,0], p_s[:,1],'rs',  markersize = 4, label='speed')
plt.plot(p_sx[:,0], p_sx[:,1],'gs', markersize = 4, label='speed_x')
plt.legend(loc='upper right')

Let's plot the acceleration vs time. Keep in mind that it should be constant while the ball flies, at g = 10.72 yards/(seconds squared), the gravitational acceleration, or acceleration of falling bodies.
Instead of a constant acceleration, the values are all over the place, only around t=1s it seems to keep a constant value which is quite above 10.7 yards/(seconds squared). 
What happened at around 2 seconds? Why did the acceleration suddenly increase? The souting data indicates a hang time of 3.85s.

In [ ]:
plt.plot(p_a[:,0], p_a[:,1],'bo', label='acceleration', markersize = 4 )
plt.legend(loc='upper right')

Here I plotted all three together, speed, calculated speed along x and acceleration. 

In [ ]:
plt.plot(p_s[:,0], p_s[:,1],'rs',  markersize = 4, label='speed')
plt.plot(p_sx[:,0], p_sx[:,1],'gs', markersize = 4, label='speed_x')
plt.plot(p_a[:,0], p_a[:,1],'bo', label='acceleration', markersize = 4 )
plt.legend(loc='upper right')


Questions: 
Why does the acceleration change while the ball is in the air? It should stay constant.
Why does the horizontal speed change while the ball is in the air? It should stay constant.
According to the scouting data the ball was in the air up to 3.85s after the kick. 


Let's do the same analysis for the 56 yards punt that follows

In [ ]:
tr_ball_2 = tracking2018.query('gameId == 2018090600 & playId == 366 & team == "football"').reset_index(drop = True)

In [ ]:
tr_ball.query('event == "punt"')['gtime'].reset_index(drop = True)

In [ ]:
tr_ball = tr_ball_2
tr_ball['gtime'] = tr_ball['time'].str.split(":").str[-1].astype('float32')
T0 = tr_ball.query('event == "punt"')['gtime'].reset_index(drop = True)[0] 
tr_ball['gtime'] = tr_ball['gtime'] - T0

In [ ]:
# here I add a column with the calculated speed in the x direction
tr_ball['sx'] = tr_ball['s'] 
for i in range(1,len(tr_ball)):
    tr_ball.at[i,'sx'] = (tr_ball.at[i,'x'] - tr_ball.at[i-1,'x'])/(tr_ball.at[i,'gtime'] - tr_ball.at[i-1,'gtime'])
    


In [ ]:
# ready for plotting, let's just turn all these into numpy arrays
#acceleration
p_a = tr_ball[['gtime', 'a']].to_numpy()
#speed
p_s = tr_ball[['gtime', 's']].to_numpy()
#calculated speed along x
p_sx = tr_ball[['gtime', 'sx']].to_numpy()


In [ ]:
# let's plot s and s_x

plt.plot(p_s[:,0], p_s[:,1],'rs',  markersize = 4, label='speed')
plt.plot(p_sx[:,0], p_sx[:,1],'gs', markersize = 4, label='speed_x')
plt.legend(loc='upper right')

In [ ]:
plt.plot(p_a[:,0], p_a[:,1],'bo', label='acceleration', markersize = 4 )
plt.legend(loc='upper right')

In [ ]:
plt.plot(p_s[:,0], p_s[:,1],'rs',  markersize = 4, label='speed')
plt.plot(p_sx[:,0], p_sx[:,1],'gs', markersize = 4, label='speed_x')
plt.plot(p_a[:,0], p_a[:,1],'bo', label='acceleration', markersize = 4 )
plt.legend(loc='upper right')
